In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import rampy as rp
from utils.PCA_tool import PCA_transform
from joblib import dump
import warnings
warnings.filterwarnings('ignore')
from aggmap import AggMap

In [2]:
def find_rrs_csv_files(directory):
    rrs = []
    for root, _, files in os.walk(directory):
        for f in files:
            if f.endswith('.csv') and 'rrs' in f:
                rrs.append(f)
    return rrs

def extract_data(ID, file_index, rrs_files, folder_path):
    csv_path = os.path.join(folder_path, rrs_files[file_index])
    data = pd.read_csv(csv_path, skiprows=[0], header=0, low_memory=False)
    n = len(data)
    if n == 307:
        rows = range(3, 103)          
    elif n == 157:
        rows = range(3, 53)          
    data_tmp = data.iloc[list(rows)].copy()
    data_tmp['ID_num'] = [f'{ID}_{i}' for i in rows]  
    return data_tmp

In [3]:
load_plan = [
    ('/sol_00257/', [0, 1]),#Dourbes
    ('/sol_00269/', [6, 7, 2, 3, 4, 5]),#Dourbes
    ('/sol_00207/', [0, 1]),#Garde
    ('/sol_00208/', [4, 5, 0, 1, 2, 3]),#Garde
    ('/sol_00293/', [0, 1]),#Quartier
    ('/sol_00304/', [8, 9, 6, 7, 4, 5, 2, 3]),#Quartier
    ('/sol_00162/', [0, 3]),#Guillaumes
    ('/sol_00161/', [3, 6]),#Guillaumes
    ('/sol_00349/', [0, 1]),#Montpezat
    ('/sol_00186/', [3]),#Bellegarde
    ('/sol_00370/', [0, 1]),#Alfalfa
    ('/sol_00747/', [0, 1, 2, 3]),#Solva
    ('/sol_00751/', [2, 0]),#Solva
    ('/sol_00782/', [2, 3, 0, 1]),#Solitude Lake
    ('/sol_00789/', [0, 1]),#Ouzel Falls
    ('/sol_00790/', [0, 2]),#Ouzel Falls
    ('/sol_00851/', [0, 1, 2, 3]),#Lake Haiyaha
    ('/sol_00852/', [0, 1, 3, 2]),#Lake Haiyaha
    ('/sol_00865/', [0]),#Dragon's Egg Rock
    ('/sol_00860/', [1, 0]),#Dragon's Egg Rock
    ('/sol_00861/', [1, 0]),#Dragon's Egg Rock
    ('/sol_00879/', [0, 1, 2]),#Gabletop Mountain
    ('/sol_00894/', [1, 0]),#Thunderbolt Peak
]

In [4]:
data0   = pd.DataFrame()
current_id = 1
base_path  = 'data_SHERLOC'

for subfolder, file_indices in load_plan:
    print(subfolder)
    folder = base_path + subfolder
    rrs    = find_rrs_csv_files(folder)
    for idx in file_indices:
        part = extract_data(current_id, idx, rrs, folder)
        data0 = pd.concat([data0, part], ignore_index=True)
        current_id += 1

folder_path = 'data_SHERLOC/sol_00257/'
rrs_files = find_rrs_csv_files(folder_path)
data = pd.read_csv(folder_path + rrs_files[0], skiprows=[0], header=0)
data_columns = list(data.iloc[0, :].values) + ['ID']
data0.columns = data_columns

labels = pd.read_excel('dataset/label.xlsx', engine='openpyxl')
labels = labels[labels['type'].notna()]
labels = labels[~labels['num'].astype(str).str.startswith('data')]
labels = list(labels.type)+ (data0.shape[0]-labels.shape[0])*[0]
data0['label'] = labels

/sol_00257/
/sol_00269/
/sol_00207/
/sol_00208/
/sol_00293/
/sol_00304/
/sol_00162/
/sol_00161/
/sol_00349/
/sol_00186/
/sol_00370/
/sol_00747/
/sol_00751/
/sol_00782/
/sol_00789/
/sol_00790/
/sol_00851/
/sol_00852/
/sol_00865/
/sol_00860/
/sol_00861/
/sol_00879/
/sol_00894/


In [5]:
data = data0.iloc[:, 103:479]
lambda0 = 248.6  
wavelengths = np.array(data.columns, dtype=float) 
raman_shift = np.round((1/lambda0 - 1/wavelengths) * 1e7, 1)
data.columns = raman_shift

In [6]:
def group_noise_scale_linear(df, low=2500, high=3000, group_size=50, eps=1e-8):
    cols = df.columns.astype(float).to_numpy()          
    mask = (cols >= low) & (cols <= high)
    sel = np.where(mask)[0]                 
    wn_sel = cols[sel].astype(np.float32)    
    X_all = df.to_numpy(dtype=np.float32)    
    N = X_all.shape[0]
    idx = np.arange(N)
    group_id = idx // group_size
    tail_vals = X_all[:, sel]
    X_design = np.vstack([wn_sel, np.ones_like(wn_sel)]).T 
    XT_X_inv = np.linalg.inv(X_design.T @ X_design)         
    pseudo = XT_X_inv @ X_design.T                        
    beta_all = tail_vals @ pseudo.T                        
    a = beta_all[:, 0:1]                               
    b = beta_all[:, 1:2]                                 
    baseline = a * wn_sel[None, :] + b
    noise = tail_vals - baseline
    rmsd_i = np.sqrt((noise**2).mean(axis=1))             
    group_noise = pd.Series(rmsd_i).groupby(group_id).median().to_dict()
    scales = np.array([group_noise[g] for g in group_id], dtype=np.float32)
    X_scaled = X_all / (scales[:, None] + eps)
    df_scaled = pd.DataFrame(X_scaled, index=df.index, columns=df.columns)
    return df_scaled, group_noise, rmsd_i
    
data_scaled, group_noise, rmsd_i = group_noise_scale_linear(data)

In [7]:
def remove_negative(data, method = 'linear'):   
    cleaned = []
    it = range(data.shape[0])
    it = tqdm(it, desc = 'Removing negatives', total = data.shape[0])
    for i in it:
        y = data.iloc[i].to_numpy(dtype = float).reshape(-1)
        y_nan = y.copy()
        y_nan[y_nan < 0] = np.nan
        y_interp = pd.Series(y_nan).interpolate(
            method = method, limit_direction = 'both'
        ).to_numpy()
        cleaned.append(y_interp)
    cleaned = pd.DataFrame(cleaned, index = data.index, columns = data.columns)
    return cleaned
data_positive = remove_negative(data_scaled, method = 'linear')

Removing negatives: 100%|█████████████████████████████████████████████████████████| 4500/4500 [00:01<00:00, 3366.79it/s]


In [8]:
def baseline_drpls(x, data):
    x = np.asarray(x, dtype=float).reshape(-1)
    bir = np.array([[float(x.min()), float(x.max())]], dtype = float)
    corrected, baselines = [], []
    for i in tqdm(range(data.shape[0]), desc = 'Baseline correcting (drPLS)'):
        y = data.iloc[i].to_numpy(dtype = float).reshape(-1)
        y_corr, bsl = rp.baseline( x, y, bir, method = 'drPLS')
        corrected.append(np.asarray(y_corr).reshape(-1))
        baselines.append(np.asarray(bsl).reshape(-1))
    corrected = pd.DataFrame(corrected, index=data.index, columns=data.columns)
    baselines = pd.DataFrame(baselines, index=data.index, columns=data.columns)
    return corrected, baselines
data_corrected, baselines = baseline_drpls(raman_shift, data_positive)

Baseline correcting (drPLS): 100%|██████████████████████████████████████████████████| 4500/4500 [00:49<00:00, 90.80it/s]


In [9]:
result = pd.concat([data_corrected.iloc[:,:58], data0.iloc[:, -2:]], axis = 1)
result.to_csv('dataset/data.csv', index = False)

In [10]:
X = result.iloc[:,:58]
X.columns = X.columns.astype(str)
X.shape

(4500, 58)

In [11]:
df_pca = PCA_transform(X,10)
df_pca.shape

(4500, 55)

In [12]:
mp_sar = AggMap(X, metric = 'euclidean')
mp_sar = mp_sar.fit(cluster_channels = 7, n_neighbors=15, min_dist=0.1, verbose = 0)
mp_car = AggMap(df_pca,metric = 'euclidean')
mp_car = mp_car.fit(cluster_channels = 7, n_neighbors=15, min_dist=0.1, verbose = 0, init='random')

2025-12-01 17:31:18,339 - INFO - [bidd-aggmap] - Calculating distance ...
2025-12-01 17:31:18,342 - INFO - [bidd-aggmap] - the number of process is 16


100%|##################################################################################| 58/58 [00:00<00:00, 735.66it/s]


2025-12-01 17:31:19,157 - INFO - [bidd-aggmap] - applying hierarchical clustering to obtain group information ...
2025-12-01 17:31:23,336 - INFO - [bidd-aggmap] - Applying grid assignment of feature points, this may take several minutes(1~30 min)
2025-12-01 17:31:23,346 - INFO - [bidd-aggmap] - Finished
2025-12-01 17:31:23,352 - INFO - [bidd-aggmap] - Calculating distance ...
2025-12-01 17:31:23,353 - INFO - [bidd-aggmap] - the number of process is 16


100%|##################################################################################| 55/55 [00:00<00:00, 765.13it/s]


2025-12-01 17:31:24,053 - INFO - [bidd-aggmap] - applying hierarchical clustering to obtain group information ...
2025-12-01 17:31:24,179 - INFO - [bidd-aggmap] - Applying grid assignment of feature points, this may take several minutes(1~30 min)
2025-12-01 17:31:24,187 - INFO - [bidd-aggmap] - Finished


In [14]:
X1 = mp_sar.batch_transform(X.values,scale_method = 'minmax')
dump(X1,'dataset/sherloc_x1.data')

100%|#############################################################################| 4500/4500 [00:00<00:00, 6323.59it/s]


['dataset/sherloc_x1.data']

In [15]:
X2 = mp_car.batch_transform(df_pca.values,scale_method = 'minmax')
dump(X2,'dataset/sherloc_x2.data')

100%|#############################################################################| 4500/4500 [00:00<00:00, 6920.96it/s]


['dataset/sherloc_x2.data']